<a href="https://colab.research.google.com/github/gyasifred/msc-thesis/blob/main/bleu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install -q -U tensorflow-text-nightly==2.11.0.dev20220817
!pip install -U -q tf-nightly==2.11.0.dev20220817

In [32]:
# inslall nltk
!pip install --user -U nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Import packages

In [33]:
import tensorflow as tf
import tensorflow_text
import numpy as np
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
import time

import french_twi translator from google drive

In [34]:
french_twi_translator= tf.saved_model.load('/content/drive/MyDrive/french_twi_translator')

test 1

In [35]:

print("Translation: ",french_twi_translator("Kwaku a essayé de briser le combat entre Abena et John.").numpy().decode("utf-8"))
print("Ground Truth: Kwaku bɔɔ mmɔden sɛ ɔbɛpaapae Abena ne John ntam ntɔkwaw no mu.")


Translation:  kwaku bɔɔ mmɔden sɛ obegyae ɔko a ɛkɔɔ so wɔ abena ne john ntam no .
Ground Truth: Kwaku bɔɔ mmɔden sɛ ɔbɛpaapae Abena ne John ntam ntɔkwaw no mu.


test 2

In [36]:
print("Translation: ",french_twi_translator("Asamoah a attaché la ficelle au cerf-volant").numpy().decode("utf-8"))
print("Ground Truth: Asamoah de hama no bɔɔ asangoli no mu.")

Translation:  asamoah de hama no bɔɔ asangoli no mu .
Ground Truth: Asamoah de hama no bɔɔ asangoli no mu.


In [37]:
hypothesis = french_twi_translator("Asamoah a attaché la ficelle au cerf-volant").numpy().decode("utf-8")
reference= "Asamoah de hama no bɔɔ asangoli no mu."
hypothesis=hypothesis.replace(" ' ", "'").replace(" .", ".").replace(" ?", "?").replace(" !", "!")\
            .replace(' " ', '" ').replace(' "', '"').replace(" : ", ": ").replace(" ( ", " (")\
            .replace(" ) ", ") ").replace(" , ", ", ").split()
reference = reference.lower().split()
# print out hypothesis and reference
print("hypothesis: ",hypothesis)
print("reference: ",reference)

hypothesis:  ['asamoah', 'de', 'hama', 'no', 'bɔɔ', 'asangoli', 'no', 'mu.']
reference:  ['asamoah', 'de', 'hama', 'no', 'bɔɔ', 'asangoli', 'no', 'mu.']


In [38]:
# check sentence bleu
chencherry = SmoothingFunction()
sentence_bleu([reference],hypothesis,smoothing_function=chencherry.method2)

1.0

calculate blue for list of sentence

In [39]:
hypothesis = ["Vous devriez parler au professeur vous meme .",
              "Je conduis une voiture noire .",
              "Avez vous apprecie le spectacle ?",
              "Ces chemises sont identiques .",
              "Elle a quelque chose dans la main .",
              "Il est gentil aujourd hui .",
              "Il y avait beaucoup de garcons et de filles dans le parc .",
              "Je veux vous poser une question .",
              "Es tu serieux ?"]

              
reference= ["Ɛsɛ sɛ w’ankasa wo ne ɔkyerɛkyerɛfo no kasa.",
            "Meka kar tuntum bi.",
            "So w’ani gyee ɔyɛkyerɛ no ho?",
            "Saa ntade yi yɛ pɛ.",
            "Ɔwɔ biribi wɔ ne nsam.",
            "Ɔyɛ obi a ne ho yɛ fɛ nnɛ.",
            "Ná mmarimaa ne mmeawa pii wɔ abɔnten so.",
            "Mepɛ sɛ mibisa wo asɛm bi .",
            "So woyɛ aniberesɛm ?"]

In [40]:
weights = [(1./2,1./2),(1./3,1./3,1./3),(1./4,1./4,1./4,1./4)]
smooth= SmoothingFunction()
bleu_total=np.array([0.,0.,0.])
length = len(hypothesis)
for i in range(length):
  hypothesis[i] = hypothesis[i][:-1]
  reference[i]=reference[i][:-1]
  groundtruth= reference[i].lower().split()
  groundtruth=[groundtruth]
  translated_text=french_twi_translator(hypothesis[i]).numpy().decode("utf-8")
  print("Translated Text: ",translated_text)
  print("Ground Truth: ", reference[i])
  candidate=translated_text.replace(" ' ", "'").replace(" .", ".").replace(" ?", "?").replace(" !", "!")\
            .replace(' " ', '" ').replace(' "', '"').replace(" : ", ": ").replace(" ( ", " (")\
            .replace(" ) ", ") ").replace(" , ", ", ").split()
  bleu=np.array(sentence_bleu(groundtruth, candidate,weights,smooth.method2))
  bleu_total += bleu
print()
print("The BLEU SCORE for 2 grams, three grams, and four grams are: ",bleu_total/length)
  

Translated Text:  ɛsɛ sɛ misua sɛ ɔka franse kasa .
Ground Truth:  Ɛsɛ sɛ w’ankasa wo ne ɔkyerɛkyerɛfo no kasa
Translated Text:  m adwenem yɛ tuntum .
Ground Truth:  Meka kar tuntum bi
Translated Text:  wobuu no ma wiase no .
Ground Truth:  So w’ani gyee ɔyɛkyerɛ no ho
Translated Text:  saa apɔnsono yi teɛsono wɔn mu biara .
Ground Truth:  Saa ntade yi yɛ pɛ
Translated Text:  ɔfaa biribi kyerɛɛ no .
Ground Truth:  Ɔwɔ biribi wɔ ne nsam
Translated Text:  ɔyɛ obi a ɔwɔ nyansa nnɛ .
Ground Truth:  Ɔyɛ obi a ne ho yɛ fɛ nnɛ
Translated Text:  na mmea fitaa bi wɔ mmoa yɛmmea hɔ .
Ground Truth:  Ná mmarimaa ne mmeawa pii wɔ abɔnten so
Translated Text:  mepɛ sɛ mibisa wo asɛm bi kɛkɛ .
Ground Truth:  Mepɛ sɛ mibisa wo asɛm bi 
Translated Text:  wuntumi nyɛ anibere wom .
Ground Truth:  So woyɛ aniberesɛm 

The BLEU SCORE for 2 grams, three grams, and four grams are:  [0.23872942 0.23327982 0.23288317]


Calculating BLEU from file

In [41]:
# to check how fast our translator takes to translate the sentence
# to be compare with google translate later on. 
# import file for test and the referenc
# open french test document to be translated
f = open("/content/test_fr.txt", "r")
hypothesis = f.readlines()
f.close()
hypothesis= hypothesis[:300]
# open Twi test document for reference
f = open("/content/test_twi.txt", "r")
reference= f.readlines()
f.close()
reference = reference[:300]
# check the length of our input sentence
length = len(hypothesis)
# set up weight for 2-grams, 3-grams and 4-grams
weights = [(1./2,1./2),(1./3,1./3,1./3),(1./4,1./4,1./4,1./4)]
# add smoothing function -> optional
smooth= SmoothingFunction()
bleu_total=np.array([0.,0.,0.])
for i in range(length):
  hypothesis[i] = hypothesis[i]
  reference[i]=reference[i]
  groundtruth= reference[i].lower().split()
  groundtruth=[groundtruth]
  translated_text=french_twi_translator(hypothesis[i]).numpy().decode("utf-8")
  print("Translated Text: ",translated_text)
  print("Ground Truth: ", reference[i])
  candidate=translated_text.replace(" ' ", "'").replace(" .", ".").replace(" ?", "?").replace(" !", "!")\
            .replace(' " ', '" ').replace(' "', '"').replace(" : ", ": ").replace(" ( ", " (")\
            .replace(" ) ", ") ").replace(" , ", ", ").split()
  bleu=np.array(sentence_bleu(groundtruth, candidate,weights,smooth.method2))
  bleu_total += bleu
  
print()
print("The BLEU SCORE for 2 grams, three grams, and four grams are: ",bleu_total/length)
  

Translated Text:  mepɛ sɛ mihu nea enti a woyɛ saa .
Ground Truth:  Mepɛ sɛ mihu nea enti a woyɛ saa .

Translated Text:  migye di sɛ wo rentwɛn sɛ me nsa retwɛn .
Ground Truth:  Migye di sɛ yɛamma w ani ampa wo so .

Translated Text:  asamoah de nkogu asodi no too so .
Ground Truth:  Asamoa antwa sɔhwɛ no

Translated Text:  ɛyɛɛ me yaw ?
Ground Truth:  So ɛhaw wo ?

Translated Text:  masusuw ho sɛ yɛbɛbɔ mmɔden bio .
Ground Truth:  Ma yɛmfa no sɛ serew .

Translated Text:  fa aduru no bi ma me .
Ground Truth:  Fa aduru no .

Translated Text:  ɛda so ara dii ɛfrɛ so da biara .
Ground Truth:  Nantew yiye da biara da .

Translated Text:  dɛn nti na wodɔ me di saa ?
Ground Truth:  Dɛn nti na woreka eyi nyinaa akyerɛ me ?

Translated Text:  te sɛ nea asamoah ntumi nhu no mu biara nni hɔ .
Ground Truth:  Asamoah ntumi nkenkan ade wɔ sum no mu .

Translated Text:  ɛsɛ sɛ woka franse kasa koro no ara .
Ground Truth:  Ɛsɛ sɛ w ankasa wo ne ɔkyerɛkyerɛfo no kasa .

Translated Text:  m adwenem y

create a class bleu

In [42]:
import nltk
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction


class Bleu():
    def __init__(self, translator):
        self.translator = translator

    def get_bleuscore(self, testfile, referencefile, smothingfunction=None):
        if type(testfile) == str and type(referencefile) == str:
            # Open test file and read lines
            f = open(testfile, "r")
            hypothesis = f.readlines()
            f.close()
            # open refernce file and read lines
            f = open(referencefile, "r")
            reference = f.readlines()
            f.close()
        elif type(testfile) == list and type(referencefile) == list:
            hypothesis = testfile
            reference = referencefile
        else:
            print(f'File must be txt or python list')

        # check the length of our input sentence
        length = len(hypothesis)
        bleu_total = np.array([0., 0., 0.])
        weights = [(1./2, 1./2), (1./3, 1./3, 1./3),
                   (1./4, 1./4, 1./4, 1./4)]
        for i in range(length):
            hypothesis[i] = hypothesis[i]
            reference[i] = reference[i]
            groundtruth = reference[i].lower().split()
            groundtruth = [groundtruth]
            translated_text = self.translator(
                hypothesis[i]).numpy().decode("utf-8")
            # print("Translated Text: ", translated_text)
            # print("Ground Truth: ", reference[i])
            candidate = translated_text.replace(" ' ", "'").replace(" .", ".").replace(" ?", "?").replace(" !", "!")\
                .replace(' " ', '" ').replace(' "', '"').replace(" : ", ": ").replace(" ( ", " (")\
                .replace(" ) ", ") ").replace(" , ", ", ").split()
            bleu = np.array(sentence_bleu(
                groundtruth, candidate, weights, smoothing_function=smothingfunction))
            bleu_total += bleu

        return f'2-GRAMS: {bleu_total[0]/length}',f'3-GRAMS: {bleu_total[1]/length}',f'4-GRAMS: {bleu_total[2]/length}'

In [43]:
smooth= SmoothingFunction()

In [44]:
bleu = Bleu(french_twi_translator)

In [45]:
bleu.get_bleuscore(hypothesis,reference,smooth.method2)

('2-GRAMS: 0.2021367894280841',
 '3-GRAMS: 0.18654885540249508',
 '4-GRAMS: 0.17912136984403523')